# Imports

In [78]:
import sys
import json
import sqlite3
from pathlib import Path

import pandas as pd
from sqlalchemy import create_engine

root = '../..' 
sys.path.append(root)
from src.tools import paths
from src.tools import data_tools as dt
from src.tools import scrape_tools as st

# SQLITE

In [ ]:
conn = sqlite3.connect('music-data.db')

In [8]:
c = conn.cursor()

In [9]:
c.execute('CREATE TABLE STONKS (date VARCHAR(10), name VARCHAR(10), price INT(10))')

In [11]:
c.execute("INSERT INTO stoNks VALUES ('2020/01/01', 'AAPL', 400),('2020/01/07', 'AMZN', 1200)")

In [12]:
conn.commit()

In [16]:
c.execute("SELECT * from STONKS")

In [17]:
c.fetchall()

[('2020/01/01', 'AAPL', 400), ('2020/01/07', 'AMZN', 1200)]

In [28]:
pd.read_sql_query('select * from stonks',conn)

,date,name,price
0,2020/01/01,AAPL,400
1,2020/01/07,AMZN,1200


# SQL Alchemy

In [142]:
engine = create_engine('sqlite:////mnt/c/Users/Greg Mangan/code/music-ind/music-data2.db', echo=False)

In [143]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

## Songs table

In [144]:
from sqlalchemy import Column, Integer, String
class Song(Base):
    __tablename__ = 'songs'
    __table_args__ = {'extend_existing': True}
    
    id = Column(String, primary_key=True)
    title = Column(String)
    url = Column(String)
    release_date = Column(String)
    genres = Column(String)
    
    def __repr__(self):
        return f"<Song(title='{self.title}' url='{self.url}')"

In [145]:
Base.metadata.create_all(engine)

In [146]:
from sqlalchemy.orm import sessionmaker

In [147]:
Session = sessionmaker(bind=engine)
session = Session()

In [106]:
session.add(bbp)
session.commit()

In [107]:
t1 = session.query(Song).first()
print(t1)

<Song(title='boom boom pow.mp3' url='/wiki/Boom_Boom_Pow')


In [82]:
song_paths = list(paths.SONGS.glob('*.json'))


[PosixPath('/mnt/c/Users/Greg Mangan/code/music-ind/data/songs/0005648762.json'),
 PosixPath('/mnt/c/Users/Greg Mangan/code/music-ind/data/songs/0009def3d2.json'),
 PosixPath('/mnt/c/Users/Greg Mangan/code/music-ind/data/songs/001224dc29.json'),
 PosixPath('/mnt/c/Users/Greg Mangan/code/music-ind/data/songs/001536cb88.json'),
 PosixPath('/mnt/c/Users/Greg Mangan/code/music-ind/data/songs/001a98d73b.json')]

In [86]:
json.load(open(song_paths[0]))

{'id': '0005648762',
 'title': 'Kiss on My List',
 'url': '/wiki/Kiss_on_My_List',
 'artists': [{'name': 'Hall & Oates',
   'url': '/wiki/Hall_%26_Oates',
   'order': 0}],
 'release_date': 'January 24, 1981',
 'genres': ['Pop rock', 'blue-eyed soul'],
 'length': '4:25 (album version)  3:48 (45 version)',
 'songwriters': [{'name': 'Daryl Hall', 'url': '/wiki/Daryl_Hall', 'order': 0},
  {'name': 'Janna Allen', 'url': '/wiki/Janna_Allen', 'order': 1}],
 'producers': [{'name': 'Daryl Hall', 'url': '/wiki/Daryl_Hall', 'order': 0},
  {'name': 'John Oates', 'url': '/wiki/John_Oates', 'order': 1}]}

In [150]:
smap = lambda d: {
    'id' : d['id'],
    'title': d['title'],
    'release_date': d.get('release_date',''),
    'genres': ",".join(d.get('genres',[])),
    'url': d['url']
}

In [151]:
for spath in song_paths:
    
    session.add(Song(**smap(json.load(open(spath)))),)
session.commit()

In [152]:
pd.read_sql_table('songs',engine)

,id,title,url,release_date,genres
0,0005648762,Kiss on My List,/wiki/Kiss_on_My_List,"January 24, 1981","Pop rock,blue-eyed soul"
1,0009def3d2,Sherry,/wiki/Sherry_(song),August 1962,"Rock and roll,doo-wop"
2,001224dc29,Why Can't This Be Love,/wiki/Why_Can%27t_This_Be_Love,"February 26, 1986",
3,001536cb88,I'm Gonna Love You Just a Little More Baby,/wiki/I%27m_Gonna_Love_You_Just_a_Little_More_...,April 1973,"Disco,R&B"
4,001a98d73b,Telephone Line,/wiki/Telephone_Line_(song),21 May 1977 (UK)May 1977 (US),"Progressive pop,,soft rock,"
...,...,...,...,...,...
4743,ffd46eaabc,Sara,/wiki/Sara_(Fleetwood_Mac_song),5 December 1979,
4744,ffd689119d,I Can't Get Next to You,/wiki/I_Can%27t_Get_Next_to_You,"July 30, 1969","Funk,psychedelic soul"
4745,ffdc78eafb,Sexual Healing,/wiki/Sexual_Healing,1982 (1982),"Post-disco,soul,funk"
4746,ffe5348ef0,MacArthur Park,/wiki/MacArthur_Park_(song),April 1968,"Baroque pop,orchestral pop,psychedelic pop,pro..."


## People table

In [178]:
class Person(Base):
    __tablename__ = 'people'
    __table_args__ = {'extend_existing': True}
    
    id = Column(Integer,primary_key=True)
    song_id = Column(String)
    ptype = Column(String)
    name = Column(String)
    url = Column(String)
    order = Column(String)
    
    def __repr__(self):
        return f"<Person(name='{self.name}' type='{self.ptype}')"

/home/greg/.local/lib/python3.8/site-packages/sqlalchemy/ext/declarative/clsregistry.py:125: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Person, and will be replaced in the string-lookup table.
  util.warn(


In [179]:
Person.__table__.create(bind=engine)

In [169]:
def pmap(d):
    ''' Map a song dictionary to a list of people'''
    song_id = d['id']
    res = []
    
    for ptype in ['artists', 'songwriters', 'producers']:
        for person in d.get(ptype,[]):
            res.append({
                'song_id': song_id,
                'ptype': ptype.rstrip('s'),
                'name': person.get('name',''),
                'url': person.get('url',''),
                'order':person.get('ordrer','')
            })
    return res

In [172]:
ppl = []
for spath in song_paths:
    
    ppl += [Person(**d) for d in pmap(json.load(open(spath))) ]
ppl

[<Person(name='Hall & Oates' type='artist'),
 <Person(name='Daryl Hall' type='songwriter'),
 <Person(name='Janna Allen' type='songwriter'),
 <Person(name='Daryl Hall' type='producer'),
 <Person(name='John Oates' type='producer'),
 <Person(name='The Four Seasons' type='artist'),
 <Person(name='Van Halen' type='artist'),
 <Person(name='Eddie Van Halen' type='songwriter'),
 <Person(name='Michael Anthony' type='songwriter'),
 <Person(name='Sammy Hagar' type='songwriter'),
 <Person(name='Alex Van Halen' type='songwriter'),
 <Person(name='Mick Jones' type='producer'),
 <Person(name='Donn Landee' type='producer'),
 <Person(name='Eddie Van Halen' type='producer'),
 <Person(name='MIchael Anthony' type='producer'),
 <Person(name='Sammy Hagar' type='producer'),
 <Person(name='Alex Van Halen' type='producer'),
 <Person(name='Barry White' type='artist'),
 <Person(name='Electric Light Orchestra' type='artist'),
 <Person(name='George McCrae' type='artist'),
 <Person(name='Gary Lewis & the Playboys' t

In [181]:
session.add_all(ppl)

In [182]:
session.commit()

In [183]:
pd.read_sql_table('people',engine)

,id,song_id,ptype,name,url,order
0,1,0005648762,artist,Hall & Oates,/wiki/Hall_%26_Oates,
1,2,0005648762,songwriter,Daryl Hall,/wiki/Daryl_Hall,
2,3,0005648762,songwriter,Janna Allen,/wiki/Janna_Allen,
3,4,0005648762,producer,Daryl Hall,/wiki/Daryl_Hall,
4,5,0005648762,producer,John Oates,/wiki/John_Oates,
...,...,...,...,...,...,...
14825,14826,ffdc78eafb,songwriter,David Ritz,/wiki/David_Ritz,
14826,14827,ffe5348ef0,artist,Richard Harris,/wiki/Richard_Harris,
14827,14828,ffe721d2e2,artist,Brandy,/wiki/Brandy_Norwood,
14828,14829,ffe721d2e2,songwriter,Keith Crouch,/wiki/Keith_Crouch,


In [235]:
q = '''
    select 
        sq.*,
        RANK() over (order by sq.artist_sum desc) as artist_rank,
        RANK() over (order by sq.songwriter_sum desc) as songwriter_rank,
        RANK() over (order by sq.producer_sum desc) as producer_rank
        
       
    from
        (select 
            p.name, 
            count(distinct p.song_id) as song_count,
            sum(case when p.ptype='artist' then 1 else 0 end) as artist_sum,
            sum(case when p.ptype='songwriter' then 1 else 0 end) as songwriter_sum,
            sum(case when p.ptype='producer' then 1 else 0 end) as producer_sum
        from 
            people p
        left join 
            songs s
        on 
            p.song_id = s.id
        group by 
            p.name)
        as sq
        
        order by song_count desc
        limit 10

'''
pd.read_sql_query(q,engine)

,name,song_count,artist_sum,songwriter_sum,producer_sum,artist_rank,songwriter_rank,producer_rank
0,Max Martin,74,0,73,61,2199,1,1
1,Aubrey Graham,40,0,40,0,2199,2,1502
2,Madonna,37,37,30,28,1,4,3
3,Drake,37,37,0,0,1,3674,1502
4,Dr. Luke,37,0,5,37,2199,160,2
5,Lukasz Gottwald,36,0,36,0,2199,3,1502
6,the Beatles,33,33,0,0,3,3674,1502
7,Michael Jackson,33,28,12,10,5,37,26
8,Rihanna,31,31,1,0,4,1081,1502
9,Babyface,30,5,11,22,208,42,6
